# 1) Parent versus non-parent predictions

In [14]:
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

# Import the base dataset
filepath = "/Users/natashal/Projects/fusion_ML/features/feature_spaces/downsampled_majority_parent_non_parent_feature_set.csv"
dataset = pd.read_csv(filepath, sep=",")
dataset.head()

,ensg,is_parent,is_5_prime_parent,is_3_prime_parent,is_recurrent_parent,num_fusions,is_oncogene,is_TSG,num_cancers_downreg_TSG,is_cancer,...,max_expression,density_domains,density_INstruct_domains,density_PISA_res,density_ELM_LMs,density_ANCHOR_LMs,density_PTMs,density_PTMcode_sites,density_UB_sites,density_phospho_sites
0,ENSG00000000971,1,1,0,0,1,0,0,0,0,...,9.339995,0.015474,0.001451,0.000000,0.000000,0.001451,0.010155,0.010155,0.000000,0.000000
1,ENSG00000001626,1,1,1,1,2,0,1,7,0,...,6.515584,0.004540,0.003027,0.022700,0.003027,0.006053,0.049939,0.062046,0.001513,0.042373
2,ENSG00000001629,1,1,0,0,1,0,0,0,0,...,4.893801,0.005730,0.000000,0.000000,0.003274,0.163720,0.026195,0.000000,0.009823,0.013098
3,ENSG00000002746,1,0,1,0,1,0,0,0,1,...,4.163691,0.002410,0.000000,0.000000,0.000000,0.062368,0.002712,0.019886,0.002712,0.000000
4,ENSG00000002822,1,1,1,1,5,0,1,0,0,...,4.546134,0.003087,0.003087,0.342674,0.003087,0.077179,0.086440,0.086440,0.003087,0.080266


In [15]:
# drop the labels
features = dataset.drop(['ensg', 'is_parent', 'is_5_prime_parent', 'is_3_prime_parent', 'is_recurrent_parent', 'num_fusions'], axis=1)

# df to numpy array and split data into X and y
npArray = np.array(features)
X = npArray[:,:].astype(float)
features_considered = np.asarray(features.columns[:])

y = dataset['is_parent'].astype(int)

In [16]:
# Examine imputed matrix
print (dataset.shape)
pd.DataFrame(X).head()

(6302, 92)


,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.339995,0.015474,0.001451,0.000000,0.000000,0.001451,0.010155,0.010155,0.000000,0.000000
1,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.515584,0.004540,0.003027,0.022700,0.003027,0.006053,0.049939,0.062046,0.001513,0.042373
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.893801,0.005730,0.000000,0.000000,0.003274,0.163720,0.026195,0.000000,0.009823,0.013098
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.163691,0.002410,0.000000,0.000000,0.000000,0.062368,0.002712,0.019886,0.002712,0.000000
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.546134,0.003087,0.003087,0.342674,0.003087,0.077179,0.086440,0.086440,0.003087,0.080266


In [17]:
# Split into training and test sets
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.30)

In [18]:
# Search for good hyperparameter values
# Specify values to grid search over
# Search for good hyperparameter values
criterion = ['gini', 'entropy']
max_features = [None, 'sqrt', 'log2'] 
max_depth = [None, 40, 50, 60]
 
hyperparameters = {
    'criterion': criterion, 
    'max_features': max_features,
    'max_depth' : max_depth
}
 
# Grid search using cross-validation with 10 fold CV
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=hyperparameters, cv=10, n_jobs=4)
gridCV.fit(XTrain, yTrain)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 40, 50, 60]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [19]:
# Identify optimal hyperparameter values
best_criterion      = gridCV.best_params_['criterion']
best_max_features   = gridCV.best_params_['max_features']  
best_max_depth      = gridCV.best_params_['max_depth']  

print("The best performing criterion is: {}".format(best_criterion))
print("The best performing max_features value is: {}".format(best_max_features))
print("The best performing max_depth value is: {}".format(best_max_depth))

The best performing criterion is: entropy
The best performing max_features value is: log2
The best performing max_depth value is: None


In [20]:
# Train classifier using optimal hyperparameter values
# You could have also gotten this model out from gridCV.best_estimator_
rf = RandomForestClassifier(n_estimators=30,
                            max_features=best_max_features,
                           criterion=best_criterion,
                           max_depth=best_max_depth)
 
rf.fit(XTrain, yTrain)
rf_predictions = rf.predict(XTest)
 
print (metrics.classification_report(yTest, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, rf_predictions),3))
print (metrics.confusion_matrix(yTest, rf_predictions, labels=None))

             precision    recall  f1-score   support

          0       0.68      0.71      0.69       934
          1       0.70      0.67      0.69       957

avg / total       0.69      0.69      0.69      1891

('Overall Accuracy:', 0.691)
[[662 272]
 [312 645]]


In [21]:
np.unique(rf_predictions, return_counts=True)

(array([0, 1]), array([974, 917]))

In [22]:
#Examine feature importance
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("{}. Feature {}, which is {} ({})".format(f + 1, 
                                       indices[f], 
                                       features_considered[indices[f]],
                                       importances[indices[f]]))
    
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. Feature 49, which is avg_num_exons (0.059263110206)
2. Feature 74, which is tau (0.0493192325137)
3. Feature 75, which is mean_expression (0.0435773582601)
4. Feature 76, which is max_expression (0.039705436798)
5. Feature 82, which is density_PTMs (0.0386078077357)
6. Feature 73, which is eigenvector_centrality (0.037928609145)
7. Feature 72, which is closeness_centrality (0.0373506863493)
8. Feature 52, which is avg_disorder (0.036101648036)
9. Feature 48, which is avg_ensp_length (0.0355816402011)
10. Feature 47, which is num_isoforms (0.0330651809741)
11. Feature 77, which is density_domains (0.0321703293611)
12. Feature 51, which is avg_domain_length (0.0321446355579)
13. Feature 59, which is num_PTMs (0.0302985207606)
14. Feature 71, which is betweenness_centrality (0.0295918341375)
15. Feature 13, which is num_GOSlim_terms (0.0294719809269)
16. Feature 70, which is degree_centrality (0.0276875314433)
17. Feature 85, which is density_phospho_sites (0.025231988

NameError: name 'plt' is not defined

In [23]:
# write out feature importance
out_file = "/Users/natashal/Projects/fusion_ML/modelling/rf_feature_importance_parents_imputed_no_balancing.csv"

with open(out_file, 'wb') as outf:
    writer = csv.writer(outf)
    for f in range(X.shape[1]):
        row = indices[f], features_considered[indices[f]], importances[indices[f]]
        writer.writerow(row)

## Using single features for predicting outcome

In [35]:
# drop the labels
dataset = pd.read_csv(filepath, sep=",")
features = dataset.drop(['is_parent'], axis=1)

#features = dataset[['UB_density', 'ub_count']]
#features = dataset[['eigenvector_centralities']]
features = dataset[['UB_density', 'ub_count', 'ptm_count', 'num_lms', 'LM_density']]

# Convert dataframe to numpy array and split
# data into input matrix X and class label vector y
npArray = np.array(features)
X = npArray[:,:].astype(float)
    
#X = preprocessing.scale(X)
#features_considered = np.asarray(features.columns[:])

y = dataset['is_parent'].astype(int)

# train test split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.30)

# Grid search using cross-validation
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=hyperparameters, cv=10, n_jobs=4)
gridCV.fit(XTrain, yTrain)
 
# Identify optimal hyperparameter values
best_n_estim      = gridCV.best_params_['n_estimators']
best_max_features = gridCV.best_params_['max_features']  
 
print("The best performing n_estimators value is: {}".format(best_n_estim))
print("The best performing max_features value is: {}".format(best_max_features))

# Train classifier using optimal hyperparameter values
# You could have also gotten this model out from gridCV.best_estimator_
rf = RandomForestClassifier(n_estimators=best_n_estim,
                            max_features=best_max_features)
 
rf.fit(XTrain, yTrain)
rf_predictions = rf.predict(XTest)
 
print (metrics.classification_report(yTest, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, rf_predictions),2))
print (metrics.confusion_matrix(yTest, rf_predictions, labels=None))

The best performing n_estimators value is: 10
The best performing max_features value is: log2
             precision    recall  f1-score   support

          0       0.83      0.94      0.88      3704
          1       0.90      0.74      0.81      2737

avg / total       0.86      0.85      0.85      6441

('Overall Accuracy:', 0.85)
[[3466  238]
 [ 702 2035]]


In [31]:
rf.feature_importances_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='log2', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=53597059, splitter='best')

## Plot trees

In [81]:
from sklearn import tree
import graphviz 
from sklearn.externals.six import StringIO
import pydot 

rf = RandomForestClassifier(max_depth=3)
rf.fit(XTrain, yTrain)

dot_data = StringIO() 
tree.export_graphviz(rf.estimators_[5], out_file=dot_data, feature_names=dataset.columns) 

graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("/Users/natashal/Projects/fusion_ML/figures/tree5.pdf") 


True

In [ ]:
# predict

# 2) Recurrence versus singular events prediction

In [24]:
# Import the base dataset
filepath = "/Users/natashal/Projects/fusion_ML/features/feature_spaces/downsampled_majority_recurrent_non_recurrent_feature_set.csv"
dataset = pd.read_csv(filepath, sep=",")

features = dataset.drop(['ensg', 'is_parent', 'is_recurrent_parent', 'num_fusions', 'is_5_prime_parent', 'is_3_prime_parent'], axis=1)

# convert, process
npArray = np.array(features)
X = npArray[:,:].astype(float)
features_considered = np.asarray(features.columns[:])
y = dataset['is_recurrent_parent'].astype(int)

# Split into training and test sets
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.30)

In [25]:
# Search for good hyperparameter values
# Specify values to grid search over
# Search for good hyperparameter values
criterion = ['gini', 'entropy']
max_features = [None, 'sqrt', 'log2'] 
max_depth = [None, 40, 50, 60]
 
hyperparameters = {
    'criterion': criterion, 
    'max_features': max_features,
    'max_depth' : max_depth
}
 
# Grid search using cross-validation with 10 fold CV
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=hyperparameters, cv=10, n_jobs=4)
gridCV.fit(XTrain, yTrain)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 40, 50, 60]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [26]:
# Identify optimal hyperparameter values
best_criterion      = gridCV.best_params_['criterion']
best_max_features = gridCV.best_params_['max_features']  
best_max_depth = gridCV.best_params_['max_depth']  

print("The best performing criterion is: {}".format(best_criterion))
print("The best performing max_features value is: {}".format(best_max_features))
print("The best performing max_depth value is: {}".format(best_max_depth))

The best performing criterion is: entropy
The best performing max_features value is: log2
The best performing max_depth value is: 50


In [33]:
# Train classifier using optimal hyperparameter values
# You could have also gotten this model out from gridCV.best_estimator_
rf = RandomForestClassifier(n_estimators=30,
                            max_features=best_max_features,
                           criterion=best_criterion,
                           max_depth=best_max_depth)
 
rf.fit(XTrain, yTrain)
rf_predictions = rf.predict(XTest)
 
print (metrics.classification_report(yTest, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, rf_predictions),3))
print (metrics.confusion_matrix(yTest, rf_predictions, labels=None))

             precision    recall  f1-score   support

          0       0.54      0.66      0.60       276
          1       0.59      0.46      0.52       285

avg / total       0.57      0.56      0.56       561

('Overall Accuracy:', 0.561)
[[183  93]
 [153 132]]


In [29]:
# Examine feature importance
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("{}. Feature {}, which is {} ({})".format(f + 1, 
                                       indices[f], 
                                       features_considered[indices[f]],
                                       importances[indices[f]]))
    
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. Feature 76, which is max_expression (0.0400896106311)
2. Feature 75, which is mean_expression (0.0380069088142)
3. Feature 52, which is avg_disorder (0.0373127797968)
4. Feature 74, which is tau (0.0364262340502)
5. Feature 73, which is eigenvector_centrality (0.0361903793823)
6. Feature 48, which is avg_ensp_length (0.0354436534645)
7. Feature 72, which is closeness_centrality (0.035194590787)
8. Feature 49, which is avg_num_exons (0.0341824476315)
9. Feature 51, which is avg_domain_length (0.0340346400546)
10. Feature 82, which is density_PTMs (0.0339619430832)
11. Feature 13, which is num_GOSlim_terms (0.0323363504372)
12. Feature 58, which is num_ANCHOR_LMs (0.0306517110255)
13. Feature 77, which is density_domains (0.0304579405472)
14. Feature 59, which is num_PTMs (0.0300996036669)
15. Feature 71, which is betweenness_centrality (0.0287509563483)
16. Feature 47, which is num_isoforms (0.0276851386669)
17. Feature 70, which is degree_centrality (0.0269332111598

NameError: name 'plt' is not defined

In [30]:
# write out feature importance
import csv

out_file = "/Users/natashal/Projects/fusion_ML/modelling/rf_feature_importance_recurrence_imputed_not_balanced.csv"

with open(out_file, 'wb') as outf:
    writer = csv.writer(outf)
    for f in range(X.shape[1]):
        row = indices[f], features_considered[indices[f]], importances[indices[f]]
        writer.writerow(row)

# 3) Fusion events: Metastatic versus primary fusions

In [66]:
# Import the base dataset
filepath = "/Users/natashal/Projects/fusion_ML/features/feature_spaces/fusion_events_downsampled_majority_for_metastasis.csv"
dataset = pd.read_csv(filepath, sep=",")
features = dataset.drop(['ensg_3', 'ensg_5', 'fusion_id', 'cancer_type', 'is_metastatic'], axis=1)

# Convert dataframe to numpy array and split
# data into input matrix X and class label vector y
npArray = np.array(features)
X = npArray[:,:].astype(float)
    
#X = preprocessing.scale(X)
features_considered = np.asarray(features.columns[:])
y = dataset['is_metastatic'].astype(int)

# Split into training and test sets
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.30)

In [67]:
# Search for good hyperparameter values
# Specify values to grid search over
# Search for good hyperparameter values
criterion = ['gini', 'entropy']
max_features = [None, 'sqrt', 'log2'] 
max_depth = [None, 40, 50, 60]
 
hyperparameters = {
    'criterion': criterion, 
    'max_features': max_features,
    'max_depth' : max_depth
}
 
# Grid search using cross-validation with 10 fold CV
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=hyperparameters, cv=10, n_jobs=4)
gridCV.fit(XTrain, yTrain)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 40, 50, 60]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [68]:
# Identify optimal hyperparameter values
best_criterion      = gridCV.best_params_['criterion']
best_max_features = gridCV.best_params_['max_features']  
best_max_depth = gridCV.best_params_['max_depth']  

print("The best performing criterion is: {}".format(best_criterion))
print("The best performing max_features value is: {}".format(best_max_features))
print("The best performing max_depth value is: {}".format(best_max_depth))

The best performing criterion is: entropy
The best performing max_features value is: log2
The best performing max_depth value is: 50


In [88]:
# Train classifier using optimal hyperparameter values
# You could have also gotten this model out from gridCV.best_estimator_
rf = RandomForestClassifier(n_estimators=30,
                            max_features=best_max_features,
                           criterion=best_criterion,
                           max_depth=best_max_depth)
 
rf.fit(XTrain, yTrain)
rf_predictions = rf.predict(XTest)
 
print (metrics.classification_report(yTest, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, rf_predictions),3))
print (metrics.confusion_matrix(yTest, rf_predictions, labels=None))

             precision    recall  f1-score   support

          0       0.59      0.60      0.59        68
          1       0.56      0.55      0.56        64

avg / total       0.58      0.58      0.58       132

('Overall Accuracy:', 0.576)
[[41 27]
 [29 35]]


In [89]:
#Examine feature importance
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("{}. Feature {}, which is {} ({})".format(f + 1, 
                                       indices[f], 
                                       features_considered[indices[f]],
                                       importances[indices[f]]))
    
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. Feature 51, which is avg_domain_length_5_prime (0.0224904744338)
2. Feature 162, which is max_expression_3_prime (0.0206882429616)
3. Feature 137, which is avg_domain_length_3_prime (0.0206572308366)
4. Feature 159, which is eigenvector_centrality_3_prime (0.0205714743467)
5. Feature 82, which is density_PTMs_5_prime (0.0193023054945)
6. Feature 135, which is avg_num_exons_3_prime (0.0187379903038)
7. Feature 52, which is avg_disorder_5_prime (0.0187322651442)
8. Feature 144, which is num_ANCHOR_LMs_3_prime (0.0185888513865)
9. Feature 48, which is avg_ensp_length_5_prime (0.0180594713733)
10. Feature 76, which is max_expression_5_prime (0.0180108480876)
11. Feature 50, which is avg_num_domains_5_prime (0.0172500233611)
12. Feature 75, which is mean_expression_5_prime (0.0170459873066)
13. Feature 49, which is avg_num_exons_5_prime (0.0161981406743)
14. Feature 138, which is avg_disorder_3_prime (0.0160326030281)
15. Feature 99, which is num_GOSlim_terms_3_prime (0.

NameError: name 'plt' is not defined

In [90]:
# write out feature importance
out_file = "/Users/natashal/Projects/fusion_ML/modelling/rf_feature_importance_metastasis_balanced_downsampling.csv"

with open(out_file, 'wb') as outf:
    writer = csv.writer(outf)
    for f in range(X.shape[1]):
        row = indices[f], features_considered[indices[f]], importances[indices[f]]
        writer.writerow(row)